# Base de dados Caltech 256

- Base de dados: https://www.kaggle.com/datasets/jessicali9530/caltech256
- Código baseado em: https://sagemaker-examples.readthedocs.io/en/latest/introduction_to_amazon_algorithms/imageclassification_caltech/Image-classification-fulltraining-highlevel.html e https://sagemaker-examples.readthedocs.io/en/latest/introduction_to_amazon_algorithms/imageclassification_caltech/Image-classification-fulltraining.html

In [ ]:
import urllib.request
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import numpy as np
import os

In [ ]:
def download2(url):
    arquivo = url.split('/')
    print(arquivo)

In [ ]:
download2('http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec')

In [ ]:
def download(url):
    arquivo = url.split('/')[-1]
    #print(arquivo)
    if not os.path.exists(arquivo):
        urllib.request.urlretrieve(url, arquivo)

In [ ]:
download('http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec')

In [ ]:
download('http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec')

# Configurações do sagemaker

In [ ]:
session = sagemaker.Session()
role = get_execution_role()
bucket = 'redeneural'
subpasta_dataset = 'caltech'
s3_treinamento = 's3://{}/{}/train/'.format(bucket, subpasta_dataset)
s3_teste = 's3://{}/{}/validation/'.format(bucket, subpasta_dataset)
s3_output_location = 's3://{}/{}/output'.format(bucket, subpasta_dataset)
print('Localização da base de treinamento: ', s3_treinamento)
print('Localização da base de teste: ', s3_teste)
print('Localização do modelo final: ', s3_output_location)

In [ ]:
!aws s3 cp caltech-256-60-train.rec $s3_treinamento --quiet
!aws s3 cp caltech-256-60-val.rec $s3_teste --quiet

In [ ]:
!aws s3 cp caltech-256-60-train.rec $s3_treinamento

# Configurações e treinamento da rede neural

In [ ]:
# https://docs.aws.amazon.com/sagemaker/latest/dg/image-classification.html
# https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-sa-east-1.html
container = sagemaker.image_uris.retrieve(framework = 'image-classification', 
                                          region = boto3.Session().region_name)

In [ ]:
#output_path onde o modelo será salvo
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html
rede_neural = sagemaker.estimator.Estimator(image_uri = container,
                                            role = role,
                                            instance_count = 1,
                                            instance_type = 'ml.p3.2xlarge',
                                            input_mode = 'File',
                                            output_path = s3_output_location) # spot instances

In [ ]:
#num_layers = numero de camadas 
# image_shape (numeros de canais, e as dimensoes das imagens)
#num_classes existem 257 objetos diferentes na base de dados
# num_training_samples = numero total de imagens 
# mini_bath_size quantidade de lotes 
# epochs = epocas
# https://docs.aws.amazon.com/sagemaker/latest/dg/IC-Hyperparameter.html
# Resnet: https://arxiv.org/pdf/1512.03385.pdf
rede_neural.set_hyperparameters(num_layers = 18,
                                image_shape = '3,224,224',
                                num_classes = 257,
                                num_training_samples = 15420,
                                mini_batch_size = 128,
                                epochs = 15,
                                learning_rate = 0.01)

In [ ]:
train_data = sagemaker.inputs.TrainingInput(s3_treinamento,
                                            content_type = 'application/x-recordio',
                                            s3_data_type = 'S3Prefix')
test_data = sagemaker.inputs.TrainingInput(s3_teste,
                                            content_type = 'application/x-recordio',
                                            s3_data_type = 'S3Prefix')

In [63]:
data_channels = {'train': train_data, 'validation': test_data}
rede_neural.fit(inputs = data_channels, logs = True)

[04/01/2023 13:34:11 INFO 139943924692800] Epoch[0] Batch [80]#011Speed: 832.479 samples/sec#011accuracy=0.017168
[04/01/2023 13:34:14 INFO 139943924692800] Epoch[0] Batch [100]#011Speed: 856.746 samples/sec#011accuracy=0.022664
[04/01/2023 13:34:16 INFO 139943924692800] Epoch[0] Train-accuracy=0.026497
[04/01/2023 13:34:16 INFO 139943924692800] Epoch[0] Time cost=17.410
[04/01/2023 13:34:24 INFO 139943924692800] Epoch[0] Validation-accuracy=0.065848
[04/01/2023 13:34:24 INFO 139943924692800] Storing the best model with validation accuracy: 0.065848
[04/01/2023 13:34:25 INFO 139943924692800] Saved checkpoint to "/opt/ml/model/image-classification-0001.params"
[04/01/2023 13:34:27 INFO 139943924692800] Epoch[1] Batch [20]#011Speed: 942.616 samples/sec#011accuracy=0.050595
[04/01/2023 13:34:30 INFO 139943924692800] Epoch[1] Batch [40]#011Speed: 957.803 samples/sec#011accuracy=0.058117
[04/01/2023 13:34:33 INFO 139943924692800] Epoch[1] Batch [60]#011Speed: 962.571 samples/sec#011accuracy

# Deploy e previsões

In [64]:
rede_neural_classificador = rede_neural.deploy(initial_instance_count=1,
                                              instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: image-classification-2023-04-01-13-42-03-899
INFO:sagemaker:Creating endpoint-config with name image-classification-2023-04-01-13-42-03-899
INFO:sagemaker:Creating endpoint with name image-classification-2023-04-01-13-42-03-899


----------!

In [88]:
# rd para leitura
imagem = open('test.jpg', 'rb').read()

In [106]:
imagem = open('eyeglasses.jpg', 'rb').read()

In [107]:
runtime = boto3.Session().client(service_name = 'sagemaker-runtime')
resposta = runtime.invoke_endpoint(EndpointName = 'image-classification-2023-04-01-13-42-03-899',
                                   ContentType = 'application/x-image',
                                   Body = bytearray(imagem))

In [108]:
resposta

{'ResponseMetadata': {'RequestId': 'ae5fd904-33b8-4668-b4e2-38c897befa9f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ae5fd904-33b8-4668-b4e2-38c897befa9f',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Sat, 01 Apr 2023 13:54:44 GMT',
   'content-type': 'application/json',
   'content-length': '5916'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f87f37cd2d0>}

In [109]:
resultado = resposta['Body'].read()

In [110]:
resultado = json.loads(resultado)

In [102]:
sum(resultado)

1.000000537586402

In [103]:
len(resultado)

257

In [104]:
# argmax trás o indice do maior valor
probabilidade_maior = np.argmax(resultado)
probabilidade_maior

188

In [105]:
rotulo = categorias[probabilidade_maior]
rotulo

'snail'

In [ ]:
# lista de todos os objetos da base de dados
categorias = ["ak47", "american-flag", "backpack", "baseball-bat", "baseball-glove",
    "basketball-hoop", "bat", "bathtub", "bear", "beer-mug", "billiards", "binoculars",
    "birdbath", "blimp", "bonsai-101", "boom-box", "bowling-ball", "bowling-pin",
    "boxing-glove", "brain-101", "breadmaker", "buddha-101", "bulldozer", "butterfly",
    "cactus", "cake", "calculator", "camel", "cannon", "canoe", "car-tire",
    "cartman", "cd", "centipede", "cereal-box", "chandelier-101", "chess-board",
    "chimp", "chopsticks", "cockroach", "coffee-mug", "coffin", "coin", "comet",
    "computer-keyboard", "computer-monitor", "computer-mouse", "conch", "cormorant",
    "covered-wagon", "cowboy-hat", "crab-101", "desk-globe", "diamond-ring",
    "dice", "dog", "dolphin-101", "doorknob", "drinking-straw", "duck", "dumb-bell",
    "eiffel-tower", "electric-guitar-101", "elephant-101", "elk", "ewer-101",
    "eyeglasses", "fern", "fighter-jet", "fire-extinguisher", "fire-hydrant",
    "fire-truck", "fireworks", "flashlight", "floppy-disk", "football-helmet",
    "french-horn", "fried-egg", "frisbee", "frog", "frying-pan", "galaxy",
    "gas-pump", "giraffe", "goat", "golden-gate-bridge", "goldfish", "golf-ball",
    "goose", "gorilla", "grand-piano-101", "grapes", "grasshopper", "guitar-pick",
    "hamburger", "hammock", "harmonica", "harp", "harpsichord", "hawksbill-101",
    "head-phones", "helicopter-101", "hibiscus", "homer-simpson", "horse",
    "horseshoe-crab", "hot-air-balloon", "hot-dog", "hot-tub", "hourglass",
    "house-fly", "human-skeleton", "hummingbird", "ibis-101", "ice-cream-cone",
    "iguana", "ipod", "iris", "jesus-christ", "joy-stick", "kangaroo-101",
    "kayak", "ketch-101", "killer-whale", "knife", "ladder", "laptop-101", "lathe",
    "leopards-101", "license-plate", "lightbulb", "light-house", "lightning",
    "llama-101", "mailbox", "mandolin", "mars", "mattress", "megaphone", "menorah-101",
    "microscope", "microwave", "minaret", "minotaur", "motorbikes-101", "mountain-bike",
    "mushroom", "mussels", "necktie", "octopus", "ostrich", "owl", "palm-pilot",
    "palm-tree", "paperclip", "paper-shredder", "pci-card", "penguin", "people",
    "pez-dispenser", "photocopier", "picnic-table", "playing-card", "porcupine",
    "pram", "praying-mantis", "pyramid", "raccoon", "radio-telescope", "rainbow",
    "refrigerator", "revolver-101", "rifle", "rotary-phone", "roulette-wheel",
    "saddle", "saturn", "school-bus", "scorpion-101", "screwdriver", "segway",
    "self-propelled-lawn-mower", "sextant", "sheet-music", "skateboard", "skunk",
    "skyscraper", "smokestack", "snail", "snake", "sneaker", "snowmobile", "soccer-ball",
    "socks", "soda-can", "spaghetti", "speed-boat", "spider", "spoon", "stained-glass",
    "starfish-101", "steering-wheel", "stirrups", "sunflower-101", "superman", "sushi",
    "swan", "swiss-army-knife", "sword", "syringe", "tambourine", "teapot", "teddy-bear",
    "teepee", "telephone-box", "tennis-ball", "tennis-court", "tennis-racket",
    "theodolite", "toaster", "tomato", "tombstone", "top-hat", "touring-bike",
    "tower-pisa", "traffic-light", "treadmill", "triceratops", "tricycle", "trilobite-101",
    "tripod", "t-shirt", "tuning-fork", "tweezer", "umbrella-101", "unicorn", "vcr",
    "video-projector", "washing-machine", "watch-101", "waterfall", "watermelon", "welding-mask",
    "wheelbarrow", "windmill", "wine-bottle", "xylophone", "yarmulke", "yo-yo", "zebra",
    "airplanes-101", "car-side-101", "faces-easy-101", "greyhound", "tennis-shoes",
    "toad", "clutter"]

In [ ]:
rede_neural_classificador.delete_endpoint()